In [5]:
# TEXT SUMMARIZER FREQUENCY BASED METHOD.

In [39]:
# LibrRIES
from goose3 import Goose
from nltk import sent_tokenize
import string
import nltk
import pint
ureg=pint.UnitRegistry()
stopwords = nltk.corpus.stopwords.words('english')
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd   

In [9]:
def get_data():
    while True:
        print("1. For Url Based summarizing using wikipedia\n")
        print("2. For Input Text from your keyboard\n ")
        print("3.QUIT\n")
        try:
            choice = input("Enter your choice\n") 
            if int(choice) == 1:
                url=input("Enter the word to be searched on wikipedia \n") 
                url =f'https://en.wikipedia.org/wiki/{url}'
                try:
                    text=Goose().extract(url).cleaned_text
                    return text
                except:
                    print(" Wrong URL format")
            elif int(choice) ==2:
                text = input("Input the copied text")
                return text
            elif int(choice) ==3:
                print("bye\n") 
                break
            else:
                print(" wrong choice enetered, Exiting the Program \n")
                break
        except:
            print(" Input Not a number, Quitting the program")
            break

In [181]:
def preprocess_data(text):
    text=text.replace('\n','')
    text=text.lower() 
    text=sent_tokenize(text)
    save_text = text 
    translator = str.maketrans('','',string.punctuation + '0123456789'+'•°'+'─')
    clean_text=[]
    for i in text:
        a=i.translate(translator)
        clean_text.append(a) 
    # so we have punctuation free etxt here, now we need to remove stopwords and units from it.
    cleaner_text=[]
    for i in clean_text:
        l=[]
        for j in i.split():
            if j not in stopwords+list(ureg):
                l.append(j)
        cleaner_text.append(' '.join(l)) 
    # From the entire article I am only interseted inwords less than 20000.
    text = ' '.join(cleaner_text)    
    tokens=text.split()
    print(f"Total data that we are taking in consideration is {len(tokens)}\n") 
    print(f" Unique words in our data is {len(set(tokens))}\n")
    tokenizer= Tokenizer()
    tokenizer.fit_on_texts(tokens)
    tokenized_text=tokenizer.texts_to_sequences(tokens)# Giving the clean text number represenattion
    # but for frequency based summariziation we only need the frequency count from tokenizer.
    freq_dataframe = pd.DataFrame(list(tokenizer.word_counts.items()),columns=['Words','Count'])
    freq_dataframe['Weights']= freq_dataframe['Count']/max(freq_dataframe['Count'])
    return freq_dataframe,text,save_text 
# here we are sending the frequency of accurance of each word in the data.

In [46]:
text1 = get_data()

1. For Url Based summarizing using wikipedia

2. For Input Text from your keyboard
 
3.QUIT

Enter your choice
1
Enter the word to be searched on wikipedia 
chocolate


In [182]:
fdf,cleaned_text,text=preprocess_data(text1) 
# getting the frequency dataframe and original text that we have. 
# I can use the cleaned text for training 

Total data that we are taking in consideration is 3816

 Unique words in our data is 1762



In [183]:
cleaned_text# we can use this cleaned text later on for Next word prediction in a blog based program, but this is not summarized
# so I have to create a summary and therefore I will be limiting this text to 20000 words for next word prediction.

'chocolate food made roasted ground cacao seed kernels available liquid solid paste either flavoring agent foods cacao consumed form since least olmec civilization thth bce majority mesoamerican people including maya aztecs made chocolate beverages seeds cacao tree intense bitter taste must fermented develop flavor fermentation seeds dried cleaned roasted shell removed produce cocoa nibs ground cocoa mass unadulterated chocolate rough form cocoa mass liquefied heating called chocolate liquor liquor may also cooled processed two components cocoa solids cocoa butter baking chocolate also called bitter chocolate contains cocoa solids cocoa butter varying proportions without added sugar powdered baking cocoa contains fiber cocoa butter processed alkali produce dutch cocoa much chocolate consumed today form sweet chocolate combination cocoa solids cocoa butter added vegetable oils sugar milk chocolate sweet chocolate additionally contains milk powder condensed milk white chocolate contains 

# Frequency Based Text-Summarizing

In [136]:
final_text_withscores=[]
scores=[]
for i in text:# Text is currently a list of cleaned sentences
    m=0
    for j in i.split():
        if fdf[fdf['Words']==j].empty == False:
            m=m+ fdf[fdf['Words']==j]['Weights'].values[0]
    scores.append(m) 
    final_text_withscores.append(i+" "+str(m)) 
    # here we are able to get the scores of each sentence 

In [133]:
final_text_withscores # We now have the scores based of each sentence in the corpus.

['chocolate is a food made from roasted and ground cacao seed kernels that is available as a liquid, solid, or paste, either on its own or as a flavoring agent in other foods. 1.2447916666666665',
 'cacao has been consumed in some form since at least the olmec civilization (19th-11th century bce),[1][2] and the majority of mesoamerican people ─ including the maya and aztecs ─ made chocolate beverages. 1.4166666666666667',
 '[3]the seeds of the cacao tree have an intense bitter taste and must be fermented to develop the flavor. 0.22395833333333334',
 'after fermentation, the seeds are dried, cleaned, and roasted. 0.036458333333333336',
 'the shell is removed to produce cocoa nibs, which are then ground to cocoa mass, unadulterated chocolate in rough form. 2.0885416666666665',
 'once the cocoa mass is liquefied by heating, it is called chocolate liquor. 1.5729166666666665',
 'the liquor may also be cooled and processed into its two components: cocoa solids and cocoa butter. 1.21875',
 'b

In [123]:
summ_df=pd.DataFrame(text,columns=['Text']) 

In [138]:
summ_df['Scores']=scores/sum(scores)

In [153]:
summ_df# Here we have the probablity base dof each sentence in the text corpus, we have to maintain the order of these dataframes
sort_df=summ_df.copy()

In [154]:
sort_df.sort_values(by='Scores',inplace=True,ascending=False)

In [155]:
sort_df.reset_index(inplace=True,drop=True)

In [156]:
sort_df # Sorted df based of highest score

,Text,Scores
0,a sample cup is filled with the chocolate and ...,0.028734
1,"the term is also used for combination bars, wh...",0.018456
2,"white chocolate contains cocoa butter, sugar, ...",0.017449
3,chocolate is also used in cold and hot beverag...,0.016561
4,"[20]known as ""dutch cocoa"", this machine-press...",0.014120
...,...,...
273,"isbn 978-1-84813-005-0• young, allen m. (2007).",0.000000
274,and expanded ed.).,0.000000
275,isbn 978-0-8130-3044-9• none chinchilla mazari...,0.000000
276,art and myth of the ancient maya.,0.000000


In [157]:
summ_df # original df containing almost all the text that we got from the article.

,Text,Scores
0,chocolate is a food made from roasted and grou...,0.004081
1,cacao has been consumed in some form since at ...,0.004644
2,[3]the seeds of the cacao tree have an intense...,0.000734
3,"after fermentation, the seeds are dried, clean...",0.000120
4,"the shell is removed to produce cocoa nibs, wh...",0.006846
...,...,...
273,university press of florida.,0.000034
274,isbn 978-0-8130-3044-9• none chinchilla mazari...,0.000000
275,art and myth of the ancient maya.,0.000000
276,usa: yale university.,0.000000


In [158]:
def print_summary(df,percentage_by_user):
    m=0
    for i in range(len(df)):
        if m <= percentage_by_user:
            m = m+ df.Scores[i]
            print(df.Text[i])

In [159]:
print_summary(sort_df,0.5) # here we have the frequency based of the sorted dataframe 
# we will summarize the cleaned text in this format to get the most important words in the article and then limiting the words 
# around 20000 words to trainour NLP model on it.

a sample cup is filled with the chocolate and placed in the unit which then displays or prints the results.two classic ways of manually tempering chocolate are:• working the molten chocolate on a heat-absorbing surface, such as a stone slab, until thickening indicates the presence of sufficient crystal "seeds"; the chocolate is then gently warmed to working temperature.• stirring solid chocolate into molten chocolate to "inoculate" the liquid chocolate with crystals (this method uses the already formed crystals of the solid chocolate to "seed" the molten chocolate).chocolate tempering machines (or temperers) with computer controls can be used for producing consistently tempered chocolate.
the term is also used for combination bars, which are a type of candy bars• chocolate chips are small pieces of chocolate, usually drop-like, which are meant for decoration and baking• pistoles, callets and fèves are small, coin-like or bean-like pieces of chocolate meant for baking and patisserie app

In [ ]:
# The only problem that I have is I am not able to maintain the order of the text for summary.